In [ ]:
# prompt: mount to my drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
%cd drive/MyDrive/


/content/drive/MyDrive


In [ ]:
%cd EEGProject/

/content/drive/MyDrive/EEGProject


In [ ]:
!ls

eeg_feature_smooth


In [ ]:
import numpy as np
import tensorflow as tf

import scipy
import os
from os import listdir
from scipy.io import loadmat
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras import activations

In [ ]:
from tensorflow.python.client import device_lib
# Changed import to use tf.keras.backend instead of tensorflow.compat.v1.keras.backend

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.compat.v1.Session(config=config)

print(device_lib.list_local_devices())

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6336241179213685838
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4531960138892457861
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
directories = ["./eeg_feature_smooth/{}/".format(i+1) for i in range(3)]
print(directories)
channel_coords = [['0', '0', 'AF3', 'FP1', 'FPZ', 'FP2', 'AF4', '0', '0'], ['F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8'], ['FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8'], ['T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8'], ['TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8'], ['P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8'], ['0', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', '0'], ['0', '0', 'CB1', 'O1', 'OZ', 'O2', 'CB2', '0', '0']]

channel_list = ['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2']
print(len(channel_coords), len(channel_coords[0]))
coord_dict = {}
for n in range(len(channel_list)):
    for i, l in enumerate(channel_coords):
        for j, x in enumerate(l):
            if (channel_list[n] == x):
                coord_dict[n] = (i,j)
print(coord_dict)

['./eeg_feature_smooth/1/', './eeg_feature_smooth/2/', './eeg_feature_smooth/3/']
8 9
{0: (0, 3), 1: (0, 4), 2: (0, 5), 3: (0, 2), 4: (0, 6), 5: (1, 0), 6: (1, 1), 7: (1, 2), 8: (1, 3), 9: (1, 4), 10: (1, 5), 11: (1, 6), 12: (1, 7), 13: (1, 8), 14: (2, 0), 15: (2, 1), 16: (2, 2), 17: (2, 3), 18: (2, 4), 19: (2, 5), 20: (2, 6), 21: (2, 7), 22: (2, 8), 23: (3, 0), 24: (3, 1), 25: (3, 2), 26: (3, 3), 27: (3, 4), 28: (3, 5), 29: (3, 6), 30: (3, 7), 31: (3, 8), 32: (4, 0), 33: (4, 1), 34: (4, 2), 35: (4, 3), 36: (4, 4), 37: (4, 5), 38: (4, 6), 39: (4, 7), 40: (4, 8), 41: (5, 0), 42: (5, 1), 43: (5, 2), 44: (5, 3), 45: (5, 4), 46: (5, 5), 47: (5, 6), 48: (5, 7), 49: (5, 8), 50: (6, 1), 51: (6, 2), 52: (6, 3), 53: (6, 4), 54: (6, 5), 55: (6, 6), 56: (6, 7), 57: (7, 2), 58: (7, 3), 59: (7, 4), 60: (7, 5), 61: (7, 6)}


In [ ]:
n = 24
perSample = ['de_movingAve', 'de_LDS', 'psd_movingAve', 'psd_LDS']
array = np.zeros(shape=(len(directories),len(os.listdir(directories[0])), n, 4, 8, 9, 5, 64)) # features = 4 datatypes*(8 x 9 eeg channel locs)*5 frequency bands*64 timestamps(zero padded) // trials = (3 sessions) x 15 people x 24 labels
li = []
for h, dire in enumerate(directories):
    print(dire)
    data = [loadmat(dire + file) for file in os.listdir(dire)]
    for i, bigsample in enumerate(data):
        for j in range(n):
            for k, key in enumerate(perSample):
                sample = np.transpose(np.array(bigsample[key + str(j+1)]), (0,2,1))
                sample = np.pad(sample, [(0,0), (0,0), (0, 64-sample.shape[2])])
                for l, channel in enumerate(sample):
                    array[h][i][j][k][coord_dict[l][0]][coord_dict[l][1]] = channel

print(array.shape)

./eeg_feature_smooth/1/
./eeg_feature_smooth/2/
./eeg_feature_smooth/3/
(3, 15, 24, 4, 8, 9, 5, 64)


In [ ]:
_X = array.reshape(np.prod(array.shape[0:3]), *array.shape[3:])
print(_X.shape)
X_loso = array[:,:,:,1,:,:,]
X_loso = np.transpose(X_loso, (0,1,2,6,3,4,5))
print(X_loso.shape)

(1080, 4, 8, 9, 5, 64)
(3, 15, 24, 64, 8, 9, 5)


In [ ]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
labels = {0: 'neutral', 1: 'sad', 2: 'fear', 3: 'happy'}

y = np.array(session1_label * 15 + session2_label * 15 + session3_label * 15)

print(y.shape)
y_loso = np.reshape(y, (3,15,24))
print(y_loso.shape)

(1080,)
(3, 15, 24)


In [ ]:
def crossval_loso(generate_model, n_epochs, X, y):
    cvscores = []
    for i in range(15):
        a = [x for x in range(15) if x != i]
        print(a)
        X_train = X[:,a,:,:,:,:,:]
        X_test =  X[:,[i],:,:,:,:,:]
        X_train = X_train.reshape(np.prod(X_train.shape[0:3]), *X_train.shape[3:])
        X_test = X_test.reshape(np.prod(X_test.shape[0:3]), *X_test.shape[3:])
        y_train = y[:,a,:]
        y_test = y[:, [i], :]
        y_train = y_train.flatten()
        y_test = y_test.flatten()
        print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
        model = generate_model()
        model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
        print('------------------------------------------------------------------------')
        print(f'Training for fold {i} ...')
        model.fit(X_train, y_train, epochs=n_epochs, verbose=1) # validation_split=0.2)
        scores = model.evaluate(X_test, y_test, verbose=2)
        print("Score for fold %d - %s: %.6f%%" % (i, model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1])
    print('------------------------------------------------------------------------')
    print("Avg accuracies: %.6f%% (+/- %.6f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
def crossval(generate_model, n_epochs, X_train, y_train, X_test, y_test, filename = None):
    kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=0)
    bestmodel = None
    bestAcc = 0
    cvscores = []
    fold = 1
    for train, test in kfold.split(X_train, y_train):
        model = generate_model()
        model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold} ...')
        model.fit(X_train[train], y_train[train],epochs=n_epochs, verbose=1) # validation_split=0.2)
        scores = model.evaluate(X_train[test], y_train[test], verbose=1)
        print("Score for fold %d - %s: %.2f%%" % (fold, model.metrics_names[1], scores[1]*100))
        if(scores[1] > bestAcc):
            bestAcc = scores[1]
            bestmodel = model
        cvscores.append(scores[1] * 100)
        fold += 1
    print('------------------------------------------------------------------------')
    print("Avg accuracies: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    test_loss, test_acc = bestmodel.evaluate(X_test,  y_test, verbose=2)
    print('\nTest accuracy:', test_acc)
    if filename:
        pickle.dump( bestmodel, open( filename, "wb" ) )

In [ ]:
def fully_conv():
    return tf.keras.Sequential([
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [ ]:
def single_conv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv1D(
32, 3, activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [ ]:
def LSTM():
    return tf.keras.Sequential([
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
    ])

In [ ]:
def dense():
    return tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [ ]:
def twodConv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv2D(
32, 3, padding="same", activation="relu",input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.BatchNormalization(center=True, scale=True),
    tf.keras.layers.Conv2D(
64, 3, padding="same", activation="relu",input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.BatchNormalization(center=True, scale=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [ ]:
def threedConv():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv3D(32, kernel_size=5, padding="same", input_shape=X.shape[1:]))
    model.add(tf.keras.layers.BatchNormalization(trainable=False))
    model.add(tf.keras.layers.Activation(activations.relu))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.BatchNormalization(trainable=False))
    model.add(tf.keras.layers.Activation(activations.relu))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10))
    return model

In [ ]:
def convlstm2d():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.ConvLSTM2D(16, kernel_size=5, padding="same", return_sequences = False, input_shape=X.shape[1:]))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(10))
    return model

In [ ]:
X = _X.transpose(0, 5, 1,2,3,4)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 4, 8, 9, 5)
(1080, 64, 1440)


In [ ]:
crossval(dense, 40, X_train, y_train, X_test, y_test)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.3072 - loss: 1470844800.0000
Epoch 2/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.4437 - loss: 790932544.0000
Epoch 3/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4469 - loss: 213457376.0000
Epoch 4/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5590 - loss: 222807472.0000
Epoch 5/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6152 - loss: 46939292.0000
Epoch 6/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6437 - loss: 203760624.0000
Epoch 7/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6644 - loss: 4512166.5000
Epoch 8/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6921 - loss: 42021376.0000
Epoch 9/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7331 - loss: 65496.9375
Epoch 10/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7192 - 

In [ ]:
crossval(single_conv, 40, X_train, y_train, X_test, y_test)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 64ms/step - accuracy: 0.2932 - loss: 108678040.0000
Epoch 2/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3442 - loss: 24882712.0000
Epoch 3/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3036 - loss: 8383780.0000
Epoch 4/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2946 - loss: 379305.5938
Epoch 5/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2904 - loss: 445312.4375
Epoch 6/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3138 - loss: 98079.2031
Epoch 7/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3066 - loss: 14033.7969
Epoch 8/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3265 - loss: 95659.9609
Epoch 9/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3257 - loss: 1241600.0000
Epoch 10/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3182 - loss: 512.4756
Epoch 11/4

In [ ]:
X = _X.transpose(0, 5, 2,3, 4, 1)[:,:,:,:,:,:]

X = X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3], np.prod(X.shape[4:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 8, 9, 20)


In [ ]:
crossval(threedConv, 100, X_train, y_train, X_test, y_test)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 140ms/step - accuracy: 0.2154 - loss: 1854967680.0000
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.3286 - loss: 455512352.0000
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.4369 - loss: 182327248.0000
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5227 - loss: 51801544.0000
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5391 - loss: 36779284.0000
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5750 - loss: 3951643.2500
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5663 - loss: 2970454.0000
Epoch 8/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6062 - loss: 30348734.0000
Epoch 9/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5673 - loss: 11601335.0000
Epoch 10/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy

In [ ]:
X = _X.transpose(0, 5, 2,3, 4, 1)[:,:,:,:,:,1]

X = X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3], np.prod(X.shape[4:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 8, 9, 5)


In [ ]:
crossval(convlstm2d, 30, X_train, y_train, X_test, y_test)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 158ms/step - accuracy: 0.3411 - loss: 1.7123
Epoch 2/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.4416 - loss: 1.2930
Epoch 3/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.4491 - loss: 1.2668
Epoch 4/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.4175 - loss: 1.2438
Epoch 5/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.4024 - loss: 1.2121
Epoch 6/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.4489 - loss: 1.1270
Epoch 7/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.3935 - loss: 1.1859
Epoch 8/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.5020 - loss: 1.0308
Epoch 9/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.5073 - loss: 1.0544
Epoch 10/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.5331 - loss: 1.0101
Epoch 11/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/ste

In [ ]:
crossval_loso(threedConv, 100, X_loso, y_loso)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
(1008, 64, 8, 9, 5) (1008,) (72, 64, 8, 9, 5) (72,)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


------------------------------------------------------------------------
Training for fold 0 ...
Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 92ms/step - accuracy: 0.4295 - loss: 18.7576
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6883 - loss: 0.6867
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6858 - loss: 0.6349
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7167 - loss: 0.5815
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6889 - loss: 0.5661
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6778 - loss: 0.5621
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7380 - loss: 0.5458
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7010 - loss: 0.5337
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7134 - loss: 0.5326
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7018 - loss: 0.5402
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 

3/3 - 1s - 307ms/step - accuracy: 0.7361 - loss: 0.4754
Score for fold 3 - compile_metrics: 73.611110%
[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
(1008, 64, 8, 9, 5) (1008,) (72, 64, 8, 9, 5) (72,)
------------------------------------------------------------------------
Training for fold 4 ...
Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - accuracy: 0.4460 - loss: 26.9833
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7216 - loss: 0.6215
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6660 - loss: 0.6894
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7308 - loss: 0.5465
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7243 - loss: 0.5894
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6938 - loss: 0.6009
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6967 - loss: 0.5624
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7159 - loss: 0.5297
Epoch 9/100

3/3 - 1s - 314ms/step - accuracy: 0.7361 - loss: 0.4581
Score for fold 4 - compile_metrics: 73.611110%
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]
(1008, 64, 8, 9, 5) (1008,) (72, 64, 8, 9, 5) (72,)
------------------------------------------------------------------------
Training for fold 5 ...
Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - accuracy: 0.4073 - loss: 13.4594
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6596 - loss: 0.7653
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6970 - loss: 0.6232
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6791 - loss: 0.6139
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7217 - loss: 0.5994
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7252 - loss: 0.5508
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6964 - loss: 0.5652
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7136 - loss: 0.5606
Epoch 9/100
